In [1]:
import simpy
import random
import numpy as np

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

In [3]:
router_customer = []
opr_1_break_times = []
opr_2_break_times = []
shift_number = 0
end = False

In [4]:
def service(env, opr):
    if opr==operator1:
        yield env.timeout(random.lognormvariate(M,S))
        
    elif opr==operator2:
        yield env.timeout(random.uniform(*service_range))

In [5]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        # Voice recognition and routing
        router_customer.append(self.name)
        yield env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router_customer.remove(self.name)
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request(0) as req:
        
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                return 
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done with operator %d at %g' % (self.name,
                                                        (self.operator==operator2)+1,
                                                        self.env.now))

In [6]:
#Determining breaks
def break_number():
    return np.random.poisson(8, size=2)

In [7]:
def insertionSort(operator_list):
    for i in range(1,len(operator_list)):
        val = operator_list[i]
        pos = i
        while pos>0 and operator_list[pos-1]>val:
            operator_list[pos]=operator_list[pos-1]
            pos = pos-1
        operator_list[pos]=val
    print(operator_list)

In [8]:
#Break request times and sorting
def break_time(opr_1_break, opr_2_break):
    for k in range(opr_1_break):
        opr_1_break_times.append(random.uniform(0,480))
    for k in range(opr_2_break):
        opr_2_break_times.append(random.uniform(0,480))
    insertionSort(opr_1_break_times)
    insertionSort(opr_2_break_times)

In [9]:
def shift(env):
    global shift_number
    shift_number += 1
    if not end:
        [opr_1_break, opr_2_break] = break_number()
        print(opr_1_break)
        print(opr_2_break)
        break_time(opr_1_break, opr_2_break)
        #opr_2_break_times.insert(0,44)
        #opr_2_break_times.insert(1,46) These are for test.
        #print(opr_2_break_times)
        #opr_1_break_times.insert(opr_1_break, 479)
        #opr_1_break_times.insert(opr_1_break, 485) For test
        #print(opr_1_break_times)
        for i in opr_1_break_times:
            env.process(take_a_break(env, i, operator1))
        for i in opr_2_break_times:
            env.process(take_a_break(env, i, operator2))
        yield env.timeout(480)
        print("the time %g" % env.now)
        opr_1_break_times.clear()
        opr_2_break_times.clear()
        env.process(shift(env))

In [10]:
def take_a_break(env, number, opr):
    yield env.timeout(number)
    with opr.request(1) as brk:
        yield brk
        if 480*shift_number - env.now > 0:
            if 480*shift_number - env.now >= 3:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(3)
            else:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(480*shift_number - env.now)
            print("break ended for operator %d at %g" % ((opr == operator2)+1 , env.now))

In [11]:
def customer_generator(env):
    global end
    start = True
    for i in range(1000):
        
        #Every 480 a new shift starts.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if len(router_customer) < 100:
            customer = Customer('Customer %s' %(i+1), env)
    end = True

In [12]:
env = simpy.Environment()
operator1 = simpy.PriorityResource(env, capacity = 1)
operator2 = simpy.PriorityResource(env, capacity = 1)
env.process(customer_generator(env))
env.run() 

10
7
[34.07604646614267, 44.63307524333047, 74.39519318760645, 209.12059240981336, 226.6161423659573, 255.9664319411964, 292.4094909717117, 385.08427703724, 387.8161389706391, 397.1018255104171]
[90.0478027519914, 126.55138170720205, 174.5488473525917, 349.81705835821435, 393.61533834425836, 414.16006451794135, 434.4019150966053]
[34.07604646614267, 44.63307524333047, 74.39519318760645, 209.12059240981336, 226.6161423659573, 255.9664319411964, 292.4094909717117, 385.08427703724, 387.8161389706391, 397.1018255104171, 485, 479]
Customer 1 initiated a call at 26.5136
Customer 1 is rooted to operator 2 at 33.5183
Customer 1 is assigned to the operator 2 at 33.5183
break started for operator 1 at 34.076
Customer 1 is done with operator 2 at 36.8474
break ended for operator 1 at 37.076
break started for operator 1 at 44.6331
break ended for operator 1 at 47.6331
Customer 2 initiated a call at 48.4201
Customer 3 initiated a call at 49.9759
Customer 4 initiated a call at 50.2018
Customer 4 is 

Customer 331 is done with operator 1 at 2072.6
Customer 333 is assigned to the operator 1 at 2072.6
Customer 335 initiated a call at 2083.55
Customer 335 is rooted to operator 2 at 2083.79
Customer 335 is assigned to the operator 2 at 2083.79
Customer 336 initiated a call at 2084.55
Customer 333 is done with operator 1 at 2086.49
break started for operator 1 at 2086.49
Customer 336 is rooted to operator 2 at 2087.17
break ended for operator 1 at 2089.49
Customer 335 is done with operator 2 at 2090.65
Customer 336 is assigned to the operator 2 at 2090.65
Customer 336 is done with operator 2 at 2092.65
Customer 337 initiated a call at 2094.1
Customer 337 is rooted to operator 2 at 2101.3
Customer 337 is assigned to the operator 2 at 2101.3
Customer 338 initiated a call at 2102.02
Customer 337 is done with operator 2 at 2103.37
Customer 339 initiated a call at 2104.87
Customer 340 initiated a call at 2107.15
Customer 339 is rooted to operator 2 at 2108.02
Customer 339 is assigned to the o

Customer 513 is assigned to the operator 2 at 3100.58
Customer 514 initiated a call at 3101.47
Customer 513 is done with operator 2 at 3101.93
break started for operator 1 at 3103.41
Customer 515 initiated a call at 3106.27
break ended for operator 1 at 3106.41
Customer 515 is rooted to operator 2 at 3106.73
Customer 515 is assigned to the operator 2 at 3106.73
Customer 514 is rooted to operator 2 at 3107.39
Customer 515 is done with operator 2 at 3108.73
Customer 514 is assigned to the operator 2 at 3108.73
Customer 514 is done with operator 2 at 3109.81
Customer 516 initiated a call at 3133.1
break started for operator 1 at 3139.76
break ended for operator 1 at 3142.76
Customer 517 initiated a call at 3143.49
Customer 516 is rooted to operator 2 at 3145.11
Customer 516 is assigned to the operator 2 at 3145.11
Customer 517 hangs up the call at 3147.79
Customer 518 initiated a call at 3148.13
Customer 516 is done with operator 2 at 3148.76
Customer 519 initiated a call at 3149.55
Custo

Customer 826 initiated a call at 5037.13
Customer 823 is done with operator 2 at 5039.92
Customer 825 is assigned to the operator 2 at 5039.92
Customer 826 is rooted to operator 2 at 5041.89
Customer 825 is done with operator 2 at 5046.15
Customer 826 is assigned to the operator 2 at 5046.15
Customer 826 is done with operator 2 at 5052.17
Customer 827 initiated a call at 5052.62
Customer 827 is rooted to operator 2 at 5053.56
Customer 827 is assigned to the operator 2 at 5053.56
Customer 827 is done with operator 2 at 5055.71
Customer 828 initiated a call at 5056.11
Customer 828 is rooted to operator 1 at 5056.53
Customer 828 is assigned to the operator 1 at 5056.53
Customer 829 initiated a call at 5057.27
Customer 829 is rooted to operator 2 at 5058.02
Customer 829 is assigned to the operator 2 at 5058.02
Customer 830 initiated a call at 5062.4
Customer 831 initiated a call at 5062.44
Customer 832 initiated a call at 5062.91
Customer 832 is rooted to operator 1 at 5063.25
Customer 829